## Vanilla Classifcation on Full Dataset
This notebook contains first vanilla run of several classification models on the full dataset. 

In [1]:
# import libraries
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from yellowbrick.classifier import ConfusionMatrix

/home/jupyter/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


### Data Preprocessing

In [2]:
# import data and convert to pandas dataframe
dataset_v1 = pd.read_parquet('dataset_v1.parquet.gzip', engine = 'pyarrow')

In [3]:
!ls -lh dataset_v1.parquet.gzip

-rw-r--r-- 1 jupyter jupyter 220M Jan 17 18:14 dataset_v1.parquet.gzip


In [4]:
# check if read correctly
dataset_v1.head()

,lat,lon,MaxFRP,et_500m,_1_km_16_days_EVI,_1_km_16_days_EVI2,_1_km_16_days_NDVI,_1_km_16_days_NIR_reflectance,_1_km_16_days_SWIR1_reflectance,_1_km_16_days_SWIR2_reflectance,...,_1_km_16_days_blue_reflectance,_1_km_16_days_green_reflectance,_1_km_16_days_red_reflectance,_DC,_DMC,_FFMC,_ISI,_BUI,_DSR,label
0,4.3625,96.329167,0.0,3276.500000,0.6110,0.6071,0.8460,0.3885,0.3490,0.1735,...,0.0168,0.0518,0.0324,6.153957,1.339365,43.675632,0.137625,1.721346,0.000183,fwi_low
1,4.3625,96.337500,0.0,3276.500000,0.5452,0.5392,0.8948,0.3081,0.2868,0.1350,...,0.0102,0.0379,0.0171,6.153957,1.339365,43.675632,0.137625,1.721346,0.000183,fwi_low
2,4.3625,96.345833,0.0,3276.500000,0.4867,0.4878,0.6774,0.3734,0.3358,0.1938,...,0.0340,0.0663,0.0718,6.153957,1.339365,43.675632,0.137625,1.721346,0.000183,fwi_low
3,4.3625,96.354167,0.0,33.700001,0.6779,0.6552,0.9191,0.3916,0.3820,0.1855,...,0.0143,0.0471,0.0165,6.153957,1.339365,43.675632,0.137625,1.721346,0.000183,fwi_low
4,4.3625,96.362500,0.0,27.000000,0.6779,0.6552,0.9191,0.3916,0.3820,0.1855,...,0.0143,0.0471,0.0165,6.153957,1.339365,43.675632,0.137625,1.721346,0.000183,fwi_low


In [5]:
# check class imbalance
dataset_v1['label'].value_counts()

water           30991646
fwi_extreme      9308511
burned           7813581
fwi_low          3031671
fwi_veryhigh      610905
fwi_high          410397
land              351369
fwi_moderate      211015
active_fire         1353
Name: label, dtype: int64

In [6]:
# Split data into X and y
# Save coordinates for merging at end or for feature engineering later
coord = dataset_v1.loc[:,['lat', 'lon']]
X = dataset_v1.drop(['label', 'lat', 'lon'], axis = 1)
y = dataset_v1['label']

In [7]:
# Train, Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify = y)

In [ ]:
# logistic = Pipeline(steps = [('scale', StandardScaler()), ('lr', LogisticRegression(max_iter = 1000))])

In [ ]:
# log_param_grid = {
#     'logistic__C': np.logspace(-4, 4, 4),
# }

In [8]:
# Standard Scale
# Only fit on training data
# Transform both
scaler = StandardScaler()
x_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [ ]:
# one-hot-encode y ? 

### Multiclass Logisitic Regression

In [12]:
lg = LogisticRegression(n_jobs = -1)
lg.fit(x_train, y_train)

MemoryError: 

In [ ]:
# Save trained model
from joblib import dump, load
dump(lg, 'lg_v1.joblib')

In [ ]:
y_pred_train = lg.predict(x_train)

In [ ]:
# for some reason, balanced_accuracy_score is not importing
accuracy_score(y_train, y_pred_train)

In [ ]:
y_pred_test = lg.predict(x_test)
accuracy_score(y_test, y_pred_test)

In [ ]:
cm_train = ConfusionMatrix(lg, classes =['fwi_low', 'fwi_high', 'fwi_veryhigh', 'fwi_moderate', 'fwi_extreme'])
cm_train.score(x_test, y_test)
cm_train.show()